
# DenseNet Classifier: Detecting Regions of Interest in Synthetic Signals

This example demonstrates how to use DeepPeak's DenseNet classifier to identify
regions of interest (ROIs) in synthetic 1D signals containing Gaussian peaks.

We will:
- Generate a dataset of noisy signals with random Gaussian peaks
- Build and train a DenseNet classifier to detect ROIs
- Visualize the training process and model predictions

<div class="alert alert-info"><h4>Note</h4><p>This example is fully reproducible and suitable for Sphinx-Gallery documentation.</p></div>


## Imports and reproducibility



In [ ]:
import numpy as np

from DeepPeak.machine_learning.classifier import DenseNet
from DeepPeak.signals import Kernel, SignalDatasetGenerator

np.random.seed(42)

## Generate synthetic dataset



In [ ]:
NUM_PEAKS = 3
SEQUENCE_LENGTH = 200

generator = SignalDatasetGenerator(n_samples=100, sequence_length=SEQUENCE_LENGTH)

dataset = generator.generate(
    signal_type=Kernel.GAUSSIAN,
    n_peaks=(1, NUM_PEAKS),
    amplitude=(1, 20),
    position=(0.1, 0.9),
    width=(0.03, 0.05),
    noise_std=0.1,
    categorical_peak_count=False,
    compute_region_of_interest=True,
)

## Visualize a few example signals and their regions of interest



In [ ]:
dataset.plot(number_of_samples=3)

## Build and summarize the DenseNet classifier



In [ ]:
dense_net = DenseNet(
    sequence_length=SEQUENCE_LENGTH,
    filters=(32, 64, 128),
    dilation_rates=(1, 2, 4),
    kernel_size=3,
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
dense_net.build()
dense_net.summary()

## Train the classifier



In [ ]:
history = dense_net.fit(
    dataset.signals,
    dataset.region_of_interest,
    validation_split=0.2,
    epochs=20,
    batch_size=64,
)

## Plot training history



In [ ]:
dense_net.plot_model_history()

## Predict and visualize on a test signal



In [ ]:
dense_net.plot_prediction(signal=dataset.signals[0:1, :], threshold=0.4)